python3 version: 3.7.0

print( numpy.version.version ): 1.15.1

print( matplotlib.__version__ ): 2.2.3

In [2]:
import numpy as np

# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
def load_data(filename):
    datas = []
    with open(filename, "r") as ins:
        for line in ins:
            [ x, y ] = line.rstrip().split('\t')
            datas.append( [ np.array( [1] + x.split(" "), dtype="float" ), float(y) ] )

    X, Y  = np.zeros((len(datas), 5)), np.zeros((len(datas), 1))
    for idx, data in enumerate(datas):
        X[idx], Y[idx] = data[0], data[1]
    return [X, Y]

p15_17_datas = load_data("hw1_15_train.dat")

w_0 = np.zeros((5, 1))

In [4]:
def shuffle_datas(datas):
    indices = np.random.permutation(datas[0].shape[0])
    return [ data[indices] for data in datas ]

def calc_error_num(output, Y):
    return np.sum(output != Y)

def pocket_pla( w, datas, update_limit=0 ):
    pocket, update_count, least_error_count = w.copy(), 0, datas[0].shape[0]
    X, Y = datas

    position = 0
    while(True):
        output = np.sign( X.dot(w) )
        output[ np.where( output == 0 ) ] = -1
        
        current_error_count = calc_error_num(output, Y)
        if least_error_count > current_error_count:
            least_error_count = current_error_count
            pocket = w.copy()

        if (least_error_count == 0) or \
           (update_limit != 0 and update_count >= update_limit):
            return { "w": pocket, "update count": update_count }

        error_index = np.where( output != Y )[0]
        if not error_index[ error_index >= position ].any():
            position = 0
        position = error_index[ error_index >= position ][0]
        w += Y[position, 0] * X[position:position+1, :].T

        update_count += 1

In [29]:
def draw_histogram(total):
    datas_range = range( min(total), max(total), 1 )
    plt.figure( num=None, figsize=(24, 8), dpi=80, facecolor='w', edgecolor='k')
    plt.xticks( datas_range )
    plt.xlim( [datas_range[0]-1, datas_range[-1]+1] )
    plt.hist( total, edgecolor='black', linewidth=1.2, bins=datas_range, rwidth=0.8 )
    plt.show()

experiment_num = 1126
datas, total = p15_17_datas.copy(), []
for _ in range(experiment_num):
    datas = shuffle_datas(datas)
    total.append( pocket_pla( w_0.copy(), datas )["update count"] )
print ('Average update count: %.3f' % ( sum(total)/experiment_num ))

# draw_histogram(total)

Average update count: 40.833
